In [1]:
import os
import os.path
import sys
import pandas as pd
import numpy as np

In [2]:
LOCAL_PATH = 'data/'
RAW_TRACE = 'CAP_NIC1_00931_20130727230801.dms'
CONV_FILE = 'conv.txt'

In [3]:
if not os.path.exists(LOCAL_PATH + RAW_TRACE):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
    KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, LOCAL_PATH + RAW_TRACE)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [5]:
import subprocess
if not os.path.exists(LOCAL_PATH + CONV_FILE):
    tshark_command = subprocess.Popen('tshark -r {} -q -z conv,tcp > {}'.format(LOCAL_PATH + RAW_TRACE, LOCAL_PATH + CONV_FILE), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out_data, err_data = tshark_command.communicate()
    out_data, err_data = out_data.decode('utf-8'), err_data.decode('utf-8')
    
    print('output:' + out_data)
    print('error:' + err_data)

output:
error:
** (process:20307): WARNING **: Dissector bug, protocol TDS, in packet 14442: /Users/buildslave/Documents/wireshark-2.4/osx106x64/build/epan/dissectors/packet-tcp.c:3415: failed assertion "proto_desegment && pinfo->can_desegment"

** (process:20307): WARNING **: Dissector bug, protocol TDS, in packet 28323: /Users/buildslave/Documents/wireshark-2.4/osx106x64/build/epan/dissectors/packet-tcp.c:3415: failed assertion "proto_desegment && pinfo->can_desegment"

** (process:20307): WARNING **: Dissector bug, protocol TDS, in packet 37894: /Users/buildslave/Documents/wireshark-2.4/osx106x64/build/epan/dissectors/packet-tcp.c:3415: failed assertion "proto_desegment && pinfo->can_desegment"

** (process:20307): WARNING **: Dissector bug, protocol SMPP, in packet 85588: /Users/buildslave/Documents/wireshark-2.4/osx106x64/build/epan/dissectors/packet-tcp.c:3415: failed assertion "proto_desegment && pinfo->can_desegment"

** (process:20307): WARNING **: Dissector bug, protocol DCER

In [4]:
with open(LOCAL_PATH + CONV_FILE, 'r') as f:
    tcp_flow_dict = {\
        'src':[],\
        'dst':[],\
        'back_pkt':[],\
        'back_byte':[],\
        'forw_pkt':[],\
        'forw_byte':[],\
        'tot_pkt':[],\
        'tot_byte':[],\
        'rel_start':[],\
        'duration':[]\
    }
    for line in f.readlines()[5:-1]:
        fields = line.split()
        tcp_flow_dict['src'].append(fields[0])
        tcp_flow_dict['dst'].append(fields[2])
        tcp_flow_dict['back_pkt'].append(fields[3])
        tcp_flow_dict['back_byte'].append(fields[4])
        tcp_flow_dict['forw_pkt'].append(fields[5])
        tcp_flow_dict['forw_byte'].append(fields[6])
        tcp_flow_dict['tot_pkt'].append(fields[7])
        tcp_flow_dict['tot_byte'].append(fields[8])
        tcp_flow_dict['rel_start'].append(fields[9])
        tcp_flow_dict['duration'].append(fields[10])

In [6]:
tcp_flow_df = pd.DataFrame(tcp_flow_dict)

In [7]:
tcp_flow_df.shape

(17984, 10)

In [8]:
tcp_flow_df.columns

Index(['back_byte', 'back_pkt', 'dst', 'duration', 'forw_byte', 'forw_pkt',
       'rel_start', 'src', 'tot_byte', 'tot_pkt'],
      dtype='object')

In [36]:
tcp_flow_df.describe()

,back_byte,back_pkt,dst,duration,forw_byte,forw_pkt,rel_start,src,tot_byte,tot_pkt
count,17984,17984,17984,17984,17984,17984,17984,17984,17984,17984
unique,2997,334,3775,6262,3398,345,17659,15103,4919,482
top,0,0,10.2.15.129:59532,0.0000,0,0,7.625783000,10.2.4.171:64461,2120,6
freq,3818,3818,553,775,3638,3638,3,178,387,1640


In [33]:
for sampling_percentage in range(100,10,-20):
    tcp_flow_df.sample(frac=sampling_percentage/100.0).reset_index(drop=True).to_csv('{}flow_{}%.csv'.format(LOCAL_PATH,sampling_percentage), index=False)